In [1]:
from utils import get_time_indices, get_spatial_range

In [3]:
import tiledb
f = tiledb.open("/data/iharp-customized-storage/storage/experiments_tdb", mode="r")

In [4]:
start_datetime = "2014-01-01 00:00:00"
end_datetime = "2019-12-31 23:00:00"

# max_lat = 85
# min_lat = 60
# min_lon = -70
# max_lon = -10
min_lat, max_lat, min_lon, max_lon = -15, 10, 16, 76

s, e = get_time_indices(start_time=start_datetime, end_time=end_datetime)
max_la, min_la, max_lo, min_lo = get_spatial_range(max_lat=max_lat, min_lat=min_lat, max_lon=max_lon, min_lon=min_lon)
s, e, max_la, min_la, max_lo, min_lo

(0, 52583, 400, 300, 1024, 784)

In [5]:

# f = tiledb.open("/data/iharp-customized-storage/storage/experiments_tdb", mode="r")
time_range = slice(s, e)  # Time indices (inclusive start, exclusive end)
lat_range = slice(min_la, max_la)
lon_range = slice(min_lo, max_lo)
coarse_data = f[time_range, lat_range, lon_range]["temperature"]
# f.close()
coarse_data

array([[[288.42512721, 288.42916275, 288.64708177, ..., 292.57164195,
         292.51917996, 292.69876138],
        [288.8771074 , 288.8387698 , 289.00422683, ..., 293.09424405,
         293.27382547, 293.33234076],
        [289.03651113, 289.05467105, 289.39769174, ..., 293.22338125,
         293.33435853, 293.56034863],
        ...,
        [292.79561428, 292.45057583, 292.48689566, ..., 300.18468337,
         300.13222138, 300.24521643],
        [292.87228949, 292.52725104, 292.27502995, ..., 300.26741189,
         300.36224702, 300.49743752],
        [292.99133785, 292.94089363, 292.64629939, ..., 300.55191728,
         300.6729834 , 300.75369415]],

       [[287.67653501, 287.69873047, 287.98525363, ..., 292.83193412,
         292.80772089, 293.00546223],
        [288.15071066, 288.09623091, 288.34643423, ..., 293.37269614,
         293.56034863, 293.57043747],
        [288.52601565, 288.52198011, 288.78832558, ..., 293.54420648,
         293.60675731, 293.69957467],
        ...,


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd




ds_empty

<xarray.Dataset> Size: 423kB
Dimensions:    (latitude: 101, longitude: 241, time: 52584)
Coordinates:
  * latitude   (latitude) float64 808B -15.0 -14.75 -14.5 ... 9.5 9.75 10.0
  * longitude  (longitude) float64 2kB 16.0 16.25 16.5 16.75 ... 75.5 75.75 76.0
  * time       (time) datetime64[ns] 421kB 2014-01-01 ... 2019-12-31T23:00:00
Data variables:
    *empty*

In [ ]:

ds_empty

<xarray.Dataset> Size: 423kB
Dimensions:    (latitude: 100, longitude: 240, time: 52583)
Coordinates:
  * latitude   (latitude) float64 800B -15.0 -14.75 -14.5 ... 9.25 9.5 9.75
  * longitude  (longitude) float64 2kB 16.0 16.25 16.5 ... 75.25 75.5 75.75
  * time       (time) datetime64[ns] 421kB 2014-01-01 ... 2019-12-31T22:00:00
Data variables:
    *empty*

In [ ]:
ds_empty = xr.Dataset()
ds_empty["latitude"] = np.arange(-90, 90.1, 0.25)
ds_empty["longitude"] = np.arange(-180, 180.1, 0.25)
ds_empty["time"] = pd.date_range(
    start=start_datetime,
    end=end_datetime,
    freq='h',
)
ds_empty = ds_empty.sel(
    latitude=slice(min_lat, max_lat),
    longitude=slice(min_lon, max_lon),
)

len_time, len_lat, len_lon = coarse_data.shape
ds_empty = ds_empty.isel(
    time=slice(0, len_time),
    latitude=slice(0, len_lat),
    longitude=slice(0, len_lon),
)

ds_empty["temperature"] = xr.DataArray(
    data=coarse_data,
    dims=["time", "latitude", "longitude"],
    coords={
        "time": ds_empty["time"],
        "latitude": ds_empty["latitude"],
        "longitude": ds_empty["longitude"],
    },
)
ds_empty

<xarray.Dataset> Size: 10GB
Dimensions:      (latitude: 100, longitude: 240, time: 52583)
Coordinates:
  * latitude     (latitude) float64 800B -15.0 -14.75 -14.5 ... 9.25 9.5 9.75
  * longitude    (longitude) float64 2kB 16.0 16.25 16.5 ... 75.25 75.5 75.75
  * time         (time) datetime64[ns] 421kB 2014-01-01 ... 2019-12-31T22:00:00
Data variables:
    temperature  (time, latitude, longitude) float64 10GB 288.4 288.4 ... 300.3

In [12]:
coarse_data.shape

(52583, 100, 240)

In [ ]:
import pandas as pd

df = pd.read_csv("../experiment/queries/get_raster_test_set_025H3.csv")
df[
    (df["spatial_resolution"] == 0.25)
    & (df["temporal_resolution"] == "hour")
    & (df["time_span"] == 5)
    & (df["area_persent"] == 100)
    & (pd.to_datetime(df["start_time"]) > pd.to_datetime("2017-01-01"))
]